# IBM Quantum Challenge 2020 November - Exercise 2-b

[Notebook containing the challenge exercise](https://github.com/qiskit-community/IBMQuantumChallenge2020/blob/main/exercises/week-2/ex_2b_en.ipynb)

## Dependencies

In [1]:
from qiskit import Aer, ClassicalRegister, execute, QuantumCircuit, QuantumRegister
import numpy as np

## The Inner Loop

In [2]:
def comp_clauses(qc, q_s, q_l):
    """Function to implement the mathematical solution.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    q_s : list
        Superposed qbits.
    q_l : list
        Light state qbits.
    """

    # the neighbour matrix
    A = [   [1, 1, 0, 1, 0, 0, 0, 0, 0],
            [1, 1, 1, 0, 1, 0, 0, 0, 0],
            [0, 1, 1, 0, 0, 1, 0, 0, 0],
            [1, 0, 0, 1, 1, 0, 1, 0, 0],
            [0, 1, 0, 1, 1, 1, 0, 1, 0],
            [0, 0, 1, 0, 1, 1, 0, 0, 1],
            [0, 0, 0, 1, 0, 0, 1, 1, 0],
            [0, 0, 0, 0, 1, 0, 1, 1, 1],
            [0, 0, 0, 0, 0, 1, 0, 1, 1] ]

    # for each element in A
    for i in range(9):
        for j in range(9):
            # apply CNOTs whereever an element is 1
            if A[i][j] == 1:
                qc.cx(q_s[i], q_l[j])

def get_oracle_circuit():
    """Function to implement a general oracle circuit.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit for the oracle.
    """

    # register for superposed qbits
    q_s = QuantumRegister(9, 'q_s')
    # register for light state qbits
    q_l = QuantumRegister(9, 'q_l')
    # register for ancilla qbits
    q_a = QuantumRegister(7, 'q_a')
    # register for flag qbit
    q_f = QuantumRegister(1, 'q_f')
    # quantum circuit
    qc = QuantumCircuit(q_s, q_l, q_a, q_f)

    # compute clauses
    comp_clauses(qc, q_s, q_l)

    # flag the solutions
    qc.mct(q_l, q_f, ancilla_qubits=q_a, mode='v-chain')

    # uncompute clauses
    comp_clauses(qc, q_s, q_l)

    # return quantum circuit
    return qc

def get_diffuser_gate(num):   
    """Function to implement a general diffuser gate.

    Parameters
    ----------
    num : int
        Number of superposed states.

    Returns
    -------
    D : :class:`qiskit.circuit.Gate`
        Quantum gate representing the circuit.
    """

    # num-bit register for the superposed qbits
    q_s = QuantumRegister(num)
    # (num - 2)-bit register for the ancilla qbits
    q_a = QuantumRegister(num - 3)
    # quantum circuit
    qc = QuantumCircuit(q_s, q_a)

    # apply H and X gates to all qbits
    for i in range(num):
        qc.h(i)
        qc.x(i)

    # implement controlled-Z gate
    # apply H gate to the final qbit
    qc.h(num - 1)
    # flip the final bit
    qc.mct(list(range(num - 1)), num - 1, ancilla_qubits=q_a, mode='v-chain')
    # apply H gate to the final qbit
    qc.h(num - 1)

    # apply X and H gates to all qbits
    for i in range(num):
        qc.x(i)
        qc.h(i)

    # return circuit as gate
    return qc.to_gate()

## Quantum Random Access Memory

In [3]:
def init_qRAM(qc, boards):
    """Function to implement qRAM.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    boards : list
        Sequences of light states.
    """

    # default sequence: 11 -> 01 -> 00 -> 10
    # to use lowest number of gates
    seq = [3, 1, 0, 2]

    # create connection tuples of the light states 
    con = list()
    for i in range(4):
        temp = list()
        for j in range(9):
            # initialize light states complementary to given states
            # such that the clauses return an odd sum for solution qbits
            if boards[seq[i]][j] == 0:
                temp.append((0, 1))
            else:
                temp.append((-1, -1))
        con.append(temp)

    # implement qRAM
    for i in range(4):
        for j in range(9):
            # gate synthesis to convert adjacent CCNOT operations to CNOTs
            if i < 3:
                # if the elements of either tuple are not equal
                if con[i][j][0] != con[i][j][1] and con[i + 1][j][0] != con[i + 1][j][1]:
                    # remove the first CCNOT connection
                    con[i][j] = (-1, -1)
                    # reduce the second CCNOT connection to CNOT
                    con[i + 1][j] = ((i + 1) % 2, (i + 1) % 2)
            # if a connection exists
            if con[i][j][0] != -1:
                # if CNOT connection
                if con[i][j][0] == con[i][j][1]:
                    qc.cx(con[i][j][0], 2 + 9 + j)
                # else CCNOT connection
                else:
                    qc.ccx(0, 1, 2 + 9 + j)
        # handle board number
        qc.x(i % 2)

## The 9-bit Adder

In [4]:
def rccx(qc, a, b, c):
    """Function to implement a simplified two-control Toffoli gate.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    a : int
        Index of the first control qbit.
    b : int
        Index of the second control qbit.
    c : int
        Index of the target qbit.
    """
    
    # a simplified implementation of
    # qc.cx(b, c)
    # qc.cz(a, c)
    # qc.ch(b, c)
    # by combining single-qbit operations
    qc.u(np.pi / 2, np.pi / 4, np.pi, c)
    qc.cx(b, c)
    qc.tdg(c)
    qc.cx(a, c)
    qc.t(c)
    qc.cx(b, c)
    qc.u(np.pi / 2, 0, 3 * np.pi / 4, c)

def comp_add(qc, bs, s, c):
    """Function to compute a full adder.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    bs : list
        Indices of the first, second and carry-in qbits.
    s : int
        Index of the sum qbit.
    c : int
        Index of the carry qbit.
    """
    
    # XOR of fist and second bit to sum
    qc.cx(bs[0], s)
    qc.cx(bs[1], s)

    # AND of first and second bit to carry
    rccx(qc, bs[0], bs[1], c)

    # AND of carry-in and sum to carry
    rccx(qc, bs[2], s, c)

    # flip sum via carry-in
    qc.cx(bs[2], s)

def uncomp_add(qc, bs, s, c):
    """Function to uncompute a full adder.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    bs : list
        Indices of the first, second and carry-in qbits.
    s : int
        Index of the sum qbit.
    c : int
        Index of the carry qbit.
    """

    # flip sum via carry-in
    qc.cx(bs[2], s)

    # AND of carry-in and sum to carry
    rccx(qc, bs[2], s, c)

    # AND of first and second bit to carry
    rccx(qc, bs[0], bs[1], c)
    
    # XOR of fist and second bit to sum
    qc.cx(bs[1], s)
    qc.cx(bs[0], s)

def comp_add_4bit_2bit(qc, b4s, b2s, c):
    """Function to implement a VanRentergem adder.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    b4s : list
        Indices of the 4-bit sum qbits.
    b2s : list
        Indices of the 2-bit sum qbits.
    c : int
        Index of the carry qbit.
    """
    
    # 1st bit
    qc.cx(b2s[0], b4s[0])
    qc.cswap(b4s[0], b2s[0], c)

    # 2nd bit
    qc.cx(b2s[1], b4s[1])
    qc.cswap(b4s[1], b2s[1], b2s[0])

    # 3rd and 4th bit
    qc.ccx(b2s[1], b4s[2], b4s[3])
    qc.cx(b2s[1], b4s[2])

    # 2nd bit
    qc.cswap(b4s[1], b2s[1], b2s[0])
    qc.cx(b2s[0], b4s[1])

    # 1st bit
    qc.cswap(b4s[0], b2s[0], c)
    qc.cx(c, b4s[0])

def get_add_9bit_circuit():
    """Function to compute the addition of 9 qbits.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit for the oracle.
    """

    # register for the 9 qbits to add
    q_b = QuantumRegister(9)
    # register for the sum qbits
    q_s = QuantumRegister(4)
    # register for the carry qbits
    q_c = QuantumRegister(3)
    # quantum circuit
    qc = QuantumCircuit(q_b, q_s, q_c)

    # first three bits
    comp_add(qc, [0, 1, 2], 9, 9 + 1)
    # next three bits
    comp_add(qc, [3, 4, 5], 9 + 4, 9 + 4 + 1)
    # sum up the sums
    comp_add_4bit_2bit(qc, list(range(9, 9 + 4)), list(range(9 + 4, 9 + 4 + 2)), 9 + 4 + 2)
    uncomp_add(qc, [3, 4, 5], 9 + 4, 9 + 4 + 1)
    # last three bits
    comp_add(qc, [6, 7, 8], 9 + 4, 9 + 4 + 1)
    comp_add_4bit_2bit(qc, list(range(9, 9 + 4)), list(range(9 + 4, 9 + 4 + 2)), 9 + 4 + 2)
    uncomp_add(qc, [6, 7, 8], 9 + 4, 9 + 4 + 1)

    # return quantum circuit
    return qc

## The Answer Function

In [5]:
def week2b_ans_func(boards):
    """Function to return a quantum circuit to obtain the solution for multiple sequences of light states.

    Parameters
    ----------
    boards : list
        Sequences of light states.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit to obtain the solution.
    """

    # register for the four board qbits
    q_b = QuantumRegister(2, 'q_b')
    # register for the superposed qbits
    q_s = QuantumRegister(9, 'q_s')
    # register for the light state qbits
    q_l = QuantumRegister(9, 'q_l')
    # register for ancilla qbits
    q_a = QuantumRegister(7, 'q_a')
    # register for flag qbit
    q_f = QuantumRegister(1, 'q_f')
    # classical register to measure solution
    c = ClassicalRegister(2, 'c')
    # quantum circuit
    qc = QuantumCircuit(q_b, q_s, q_l, q_a, q_f, c)

    # initialize board qbits
    for i in range(2):
        qc.h(q_b[i])

    # initialize superposed qbits
    for i in range(9):
        qc.h(q_s[i])

    # initialize the flag qbit
    qc.x(q_f[0])
    qc.h(q_f[0])

    for b in range(1):
        # initialize qRAM
        init_qRAM(qc, boards)

        # oracle-diffuser loop
        for s in range(1):
            # compute oracle
            qc.append(get_oracle_circuit().to_gate(), list(range(2, 28)))
            # compute diffuser for light states
            qc.append(get_diffuser_gate(9), list(range(2, 2 + 9)) + list(range(20, 26)))

        # compute counter for number of switches
        qc.append(get_add_9bit_circuit().to_gate(), list(range(2, 2 + 9)) + list(range(20, 27)))

        # check for counter > 3
        qc.x(20 + 2)
        qc.x(20 + 3)
        qc.ccx(20 + 2, 20 + 3, q_f)
        qc.x(20 + 3)
        qc.x(20 + 2)

        # uncompute counter for number of switches
        qc.append(get_add_9bit_circuit().inverse().to_gate(), list(range(2, 2 + 9)) + list(range(20, 27)))

        # oracle-diffuser loop for switches
        for s in range(1):
            # uncompute diffuser for light states
            qc.append(get_diffuser_gate(9), list(range(2, 2 + 9)) + list(range(20, 26)))
            # uncompute oracle
            qc.append(get_oracle_circuit().inverse().to_gate(), list(range(2, 28)))

        # uninitialize qRAM
        init_qRAM(qc, boards)

        # diffuser for boards
        # apply H and X gates to all qbits
        for i in range(2):
            qc.h(i)
            qc.x(i)
        # implement controlled-Z gate
        qc.h(1)
        qc.cx(0, 1)
        qc.h(1)
        # apply X and H gates to all qbits
        for i in range(2):
            qc.x(i)
            qc.h(i)

    # measure
    qc.measure(q_b, c)

    # return reversed bits for same endian
    return qc.reverse_bits()

## The Quantum Circuit

In [6]:
# sample sequences and solutions
Q = [   [[1,1,1, 0,0,0, 1,0,0], [1,0,1, 0,0,0, 1,1,0], [1,0,1, 1,1,1, 0,0,1], [1,0,0, 0,0,0, 1,0,0]],
        [[0,0,0, 0,0,1, 0,1,1], [0,1,0, 1,0,1, 0,1,0], [0,1,1, 1,0,1, 0,1,0], [1,1,1, 0,1,0, 1,1,0]],
        [[1,0,1, 1,0,1, 0,0,1], [0,1,0, 0,0,1, 1,1,1], [0,1,1, 1,0,0, 1,0,0], [1,0,0, 1,0,0, 1,0,0]],
        [[0,0,0, 0,1,1, 0,0,1], [0,1,0, 1,1,0, 0,0,0], [0,1,1, 1,0,0, 0,0,1], [1,0,0, 0,0,0, 1,0,1]]    ]

# selected sequences of light states
boards = Q[0]

# get the quantum circuit
qc = week2b_ans_func(boards)

# # display
# qc.draw(output='mpl')

## Simulation

In [7]:
# execute the quantum circuit on the backend
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=8000)

# obtain the maximum from the result
result = job.result()
result.get_counts()

{'00': 1999, '01': 2165, '10': 1938, '11': 1898}

## Grading

In [8]:
# dependencies for calculating cost
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# calculate cost
ur = Unroller(['u3', 'cx'])
pm = PassManager(ur)
qc_cost = pm.run(qc) 
op_dict = qc_cost.count_ops()
# given criteria to calculate cost
print('Cost: {}'.format(op_dict['u3'] + op_dict['cx'] * 10))

Cost: 7762
